In [1]:
import telebot, os
from telebot import types
from collections import defaultdict

In [2]:
token= 'enter your token'
START, NAME, PHOTO, LOCATION, CONFIRMATION, DELETE,CLEAR, DELETE_PLACE = range(8)


In [3]:
USER_STATE=defaultdict (lambda: START)
def get_state(message):
    return USER_STATE[message.chat.id]

def update_state(message, state):
    USER_STATE[message.chat.id] = state

In [4]:
places=defaultdict(lambda : [])


In [5]:
def update_place(user_id, place):
    places[user_id].append(place)



In [ ]:
bot=telebot.TeleBot(token)
current_info={}

@bot.message_handler(commands=['add'])
def handle_start(message):
    bot.send_message(chat_id=message.chat.id, text='Напиши название места')
    update_state(message, NAME)
    
@bot.message_handler(func=lambda message: get_state(message) == NAME)
def handle_name(message):
    current_info['name']=message.text
    bot.send_message(chat_id=message.chat.id, text='Пришлите фото')
    update_state(message, PHOTO)
 

@bot.message_handler(content_types=['photo'])
@bot.message_handler(func=lambda message: get_state(message) == PHOTO)
def handle_photo(message):
    try:
        file = bot.get_file(message.photo[-1].file_id)
        downloaded_file = bot.download_file(file.file_path)
        chek_dir= os.path.normpath(os.getcwd()+'/telegram bot/'+str(message.from_user.id)+'/photos')
        if not os.path.isdir(chek_dir):
            os.makedirs(chek_dir)
        path_photo=os.path.normpath(os.getcwd()+'/telegram bot/'+str(message.from_user.id)+'/'+file.file_path)
        with open(path_photo, "wb") as file1:
            file1.write(downloaded_file)
        current_info['photo']=path_photo
        bot.send_message(chat_id=message.chat.id, text='Пришлите локацию')
        update_state(message, LOCATION)
    except:
        bot.reply_to(message,f"Это не фотография")
        if 'нет' in message.text.lower():
            update_state(message, LOCATION)
   
    
@bot.message_handler(content_types=['location'])
@bot.message_handler(func=lambda message: get_state(message) == LOCATION)
def handle_location(message):
    try:
        
        current_info['location']=[message.location.latitude, message.location.longitude]
        title=current_info['name']
        bot.send_message(chat_id=message.chat.id, text=f'Сохранить? место {title}')
        update_state(message, CONFIRMATION)    
    except:
        bot.reply_to(message,f"Это не локация")
        if 'нет' in message.text.lower():
            update_state(message, CONFIRMATION)
  
    
@bot.message_handler(func=lambda message: get_state(message) == CONFIRMATION)
def handle_confirmation(message):
    if 'да' in message.text.lower():
        global current_info
        update_place(message.chat.id, current_info)
        current_info = {}
        bot.send_message(chat_id=message.chat.id, text='Сохранено')
        bot.send_message(chat_id=message.chat.id, text=f'У Вас {message.from_user.first_name} {message.from_user.last_name}\
        {len(places[message.chat.id])} места')
    else:
        current_info = {}
        bot.send_message(chat_id=message.chat.id, text='Попробуйте заново добавить место')
    update_state(message, DELETE)

@bot.message_handler(commands=['list'])
@bot.message_handler(content_types=['location', 'photo'])
def handle_message(message):
    if len(places[message.chat.id])>=1:
        for x in places[message.chat.id][:11]:
            name =x['name']
            photo=x['photo']
            lat, lon=x['location']
            img= open(photo, 'rb')
            bot.send_photo(chat_id=message.chat.id, photo=img, caption=name)
            bot.send_location(message.chat.id, lat, lon)
    else:
        bot.send_message(chat_id=message.chat.id, text=f'У вас нет сохраненных мест')

@bot.message_handler(commands=['reset'])
def handle_message(message):
    bot.send_message(chat_id=message.chat.id, text='Вы действительно хотите удалить все ваши места?')
    update_state(message, CLEAR)
     
        
@bot.message_handler(func=lambda message: get_state(message) == CLEAR)
def handle_delete(message):
    if 'да' in message.text.lower():
        
        bot.send_message(chat_id=message.chat.id, text='Удалено')
        global places
        places[message.chat.id]=[]
    else:
        bot.send_message(chat_id=message.chat.id, text='Хорошо хорошо... не буду удалять')
        update_state(message, DELETE)

@bot.message_handler(commands=['delete'])
def handle_start(message):
    pl=[]
    for x in places[message.chat.id][-1:-11:-1]:
        pl.append(x['name'])
    bot.send_message(chat_id=message.chat.id, text="Какое из перечисленных вы хотите удалить:")    
    bot.send_message(chat_id=message.chat.id, text=', '.join(pl))
    update_state(message, DELETE_PLACE)

@bot.message_handler(func=lambda message: get_state(message) == DELETE_PLACE)
def handle_delete_place(message):
    
    
    for x in range(len(places[message.chat.id])):
        if places[message.chat.id][x]['name'].lower() == message.text.lower():
            places[message.chat.id].pop(x)
            bot.reply_to(message,f"Место {message.text.capitalize()} было удалено")
            break
    
    update_state(message, DELETE)
    
    
bot.polling()